In [2]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

################CHANGE THESE AS NEEDED##############
TEMP=123
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792934/scfs'
TARGET_SPECIES='N'
####################################################


os.chdir(TARGET_DIR) 
starttime=datetime.datetime.now()


print("Begin on {}".format(starttime.ctime()))
print("\nWorking dir:\t{}".format(os.getcwd())) 
print("""

TEMP:            {}
TARGET_DIR:      {}
TARGET_SPECIES:  {}
""".format(TEMP,TARGET_DIR,TARGET_SPECIES)
     )

print()




def theta_x(xi,yi,zi,xf,yf,zf):
    return np.arcsin(zf@yi)

def theta_y(xi,yi,zi,xf,yf,zf):
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)

def normalize(vector):
    return vector/np.linalg.norm(vector)

def ntheta_x(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    return np.arcsin(zf@yi)

def ntheta_y(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)



ls = os.listdir('.') 
efgfiles = [ thing for thing in ls if thing.startswith('efg') and thing.endswith('out') ]
numfiles= len(efgfiles) 
print("There are {} efg files in this directory.".format(numfiles))
print()



efgs=[] 
skipped_files=[]
for i in range(numfiles): 
    infile='efg.{}.out'.format(i) 
    try:
        efgs.append(efg.Efg(infile))
    except FileNotFoundError:
        skipped_files.append(infile)
        #print('skipped file {}'.format(infile))

if len(skipped_files) != 0:
    print('{} files were skipped.'.format(len(skipped_files)))
    print('between {} and {}'.format(skipped_files[0], skipped_files[-1]))
    print('display variable "skipped_files" to see all files not opened.')
print("{} efg files opened without error.".format(len(efgs)))
print()



print("The indices of {} atoms are:".format(TARGET_SPECIES)) 
target_atom_labels=[] 
for label in efgs[0].atom_labels: 
    if TARGET_SPECIES in label: 
        new_index = efgs[0].atom_labels.index(label)+1 
        target_atom_labels.append(new_index) 
        sys.stdout.write(str(new_index)+' ') 

print()





av_theta_xs = [] 
av_theta_ys = [] 
av_theta_y2s = [] 
av_theta_x2s = [] 
cq_coef0s = [] 
cq_coef1s = [] 
eta_coefs = [] 
cq0s = [] 
cqprime0s = [] 
cqprime1s = [] 
eta0s = [] 
etaprimes = [] 
fq0s = [] 
fqprimes = []

for atnum in target_atom_labels: 
    efg0 = efgs[0] 
    specie=atnum 
    k=specie 
    print("\nWorking on atomic specie:\t{}".format(specie)) 
    
    
    this_nucleus=efg0.atom(k) 
    cq0=this_nucleus.cq 
    cq0s.append(cq0) 
    eta0=this_nucleus.eta
    eta0s.append(eta0)
    
    
    xi=efg0.atom(k).x
    yi=efg0.atom(k).y
    zi=efg0.atom(k).z



    theta_x_vec=[]
    theta_y_vec=[]


    print("Fetching the angles in file efg.{}.out")
    for i in range(1,901):
        try:
            nucleus=efgs[i].atom(k)
            nuc=nucleus
            xf,yf,zf=nuc.x,nuc.y,nuc.z
            args=(xi,yi,zi,xf,yf,zf)
            #print("thetaX{}\t".format(i),"\t", theta_x(xi,yi,zi,xf,yf,zf),"\t", "thetaY{}\t".format(i), theta_y(xi,yi,zi,xf,yf,zf))
            theta_x_vec.append(theta_x(*args))
            theta_y_vec.append(theta_y(*args))
            sys.stdout.write("{} \r".format(i))
        except IndexError:
            sys.stdout.write("skipped:{}\n".format(i))
        
  
                 # theta_x squareds and theta_y squareds
    theta_x2_vec=[angle**2 for angle in theta_x_vec]
    theta_y2_vec=[angle**2 for angle in theta_y_vec]

    av=np.average
    av_theta_x  = av(theta_x_vec)
    av_theta_y  = av(theta_y_vec)
    av_theta_x2 = av(theta_x2_vec)
    av_theta_y2 = av(theta_y2_vec)

    av_theta_xs.append(av_theta_x)
    av_theta_ys.append(av_theta_y)
    av_theta_x2s.append(av_theta_x2)
    av_theta_y2s.append(av_theta_y2)

    print("\nReport for atomic specie:\t{}".format(k))
    print("Working Dir:\t{}".format(os.getcwd()))
    print("<tx>:\t",av_theta_x,"\t<ty>:\t",   av_theta_y,"\n",
      "<tx^2>\t:",av_theta_x2,"\t<ty^2>:", av_theta_y2,"\n")

    print("cq0=\t{}".format(cq0))
    print("eta0=\t{}".format(eta0))

    cq_coef0 = 1 - 3/2*(av_theta_x2 + av_theta_y2)
    cq_coef0s.append(cq_coef0)
    cq_coef1 = 1 - 3/2*(av_theta_x2 + av_theta_y2 - 1/2*eta0*(av_theta_x2 - av_theta_y2))
    cq_coef1s.append(cq_coef1)

    
    
    print("==================================")
    print("target:\t"+TARGET_DIR)
    print("cwd:\t"+os.getcwd())
    print("T={}K, nuclear site: {}  =".format(TEMP,specie))
    print("==================================")


    print("1-3/2(<tx^2>+<ty^2>)\t\t={}".format(cq_coef0))
    print("1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)\t={}".format(cq_coef1))
    print()


    cqprime0=cq0*cq_coef0
    cqprime0s.append(cqprime0)
    cqprime1=cq0*cq_coef1
    cqprime1s.append(cqprime1)


    print("Cq0:\t\t{}".format(cq0))
    print("Cq' from coefficient 1-3/2(<>+<>):\t\t{}".format(cqprime0))
    print("Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):\t{}".format(cqprime1))
    nuclear_spin=1
    freq_function=nqr.f1
    print(
    "Computing NQR frequencies for spin:\t{}\nusing function {}\tand simple coefficient".format(nuclear_spin, freq_function)
    )
    print("WARNING:\t eta not adjusted.  Using eta0.")
    fq=freq_function(cqprime0,eta0)
    fq0s.append(fq)
    print("fqs={}".format(freq_function(cqprime0, eta0)))
    print()
    print()

print("##################################"*2)
print("##################################"*2)
print()
print("==================================")
print("          FINAL REPORT T={}K      ".format(TEMP))
print("cwd:\t{}".format(os.getcwd()))
print("==================================")
print("""
target_atom_labels=\t{}\n
av_theta_xs=\t{}\n    
av_theta_ys=\t{}\n  
av_theta_y2s=\t{}\n
av_theta_x2s=\t{}\n 
cq_coef0s=\t{}\n
cq_coef1s=\t{}\n
eta_coefs=\t{}\n
cq0s=\t{}\n
cqprime0s=\t{}\n
cqprime1s=\t{}\n 
eta0s=\t{}\n
etaprimes=\t{}\n
fq0s=\t{}\n 
fqprimes=\t{}\n


    """.format(
    target_atom_labels,
    av_theta_xs,
    av_theta_ys, 
    av_theta_y2s, 
    av_theta_x2s, 
    cq_coef0s, 
    cq_coef1s, 
    eta_coefs, 
    cq0s, 
    cqprime0s, 
    cqprime1s, 
    eta0s, 
    etaprimes, 
    fq0s, 
    fqprimes))





print("cq_coef0s\n---------")
for item in target_atom_labels:
    print(item, cq_coef0s[target_atom_labels.index(item)])
print()


print("cq_coef1s\n---------")
for item in target_atom_labels:
    print(item, cq_coef1s[target_atom_labels.index(item)])
print()


print("cq0s\n---------")
for item in target_atom_labels:
    print(item, cq0s[target_atom_labels.index(item)])
print()


print("cqprime0s\n---------")
for item in target_atom_labels:
    print(item, cqprime0s[target_atom_labels.index(item)])
print()


print("cqprime1s\n---------")
for item in target_atom_labels:
    print(item, cqprime1s[target_atom_labels.index(item)])
print()


print("eta0s\n---------")
for item in target_atom_labels:
    print(item, eta0s[target_atom_labels.index(item)])
print()





endtime=datetime.datetime.now()
runtime=endtime-starttime
runtime_s=runtime.total_seconds()

print("DONE\truntime is {} seconds".format(str(runtime_s)))
print(endtime.ctime())
print("Goodbye.")
print()




Begin on Mon Feb 19 15:12:08 2018

Working dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792934/scfs


TEMP:            123
TARGET_DIR:      /Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792934/scfs
TARGET_SPECIES:  N


There are 901 efg files in this directory.

901 efg files opened without error.

The indices of N atoms are:
1 3 4 6 23 25 26 28 37 39 40 42 51 53 54 56 

Working on atomic specie:	1
Fetching the angles in file efg.{}.out
skipped:812
skipped:813
skipped:814
skipped:815
skipped:816
skipped:817
skipped:818
skipped:819
skipped:820
skipped:821
skipped:822
skipped:823
skipped:824
skipped:825
skipped:826
skipped:827
skipped:828
skipped:829
skipped:830
skipped:831
skipped:832
skipped:833
skipped:834
skipped:835
skipped:836
skipped:837
skipped:838
skipped:839
skipped:840
skipped:841
skipped:842
skipped:843
skipped:844
skipped:845
skipped:846
skipped:847
skipped:848
skipped:849
skipped:850
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipp

Fetching the angles in file efg.{}.out
skipped:696
skipped:812
skipped:813
skipped:814
skipped:815
skipped:816
skipped:817
skipped:818
skipped:819
skipped:820
skipped:821
skipped:822
skipped:823
skipped:824
skipped:825
skipped:826
skipped:827
skipped:828
skipped:829
skipped:830
skipped:831
skipped:832
skipped:833
skipped:834
skipped:835
skipped:836
skipped:837
skipped:838
skipped:839
skipped:840
skipped:841
skipped:842
skipped:843
skipped:844
skipped:845
skipped:846
skipped:847
skipped:848
skipped:849
skipped:850
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipped:857
skipped:858
skipped:859
skipped:860
skipped:861
skipped:862
skipped:863
skipped:864
skipped:865
skipped:866
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
s

Fetching the angles in file efg.{}.out
skipped:696
skipped:812
skipped:813
skipped:814
skipped:815
skipped:816
skipped:817
skipped:818
skipped:819
skipped:820
skipped:821
skipped:822
skipped:823
skipped:824
skipped:825
skipped:826
skipped:827
skipped:828
skipped:829
skipped:830
skipped:831
skipped:832
skipped:833
skipped:834
skipped:835
skipped:836
skipped:837
skipped:838
skipped:839
skipped:840
skipped:841
skipped:842
skipped:843
skipped:844
skipped:845
skipped:846
skipped:847
skipped:848
skipped:849
skipped:850
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipped:857
skipped:858
skipped:859
skipped:860
skipped:861
skipped:862
skipped:863
skipped:864
skipped:865
skipped:866
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
s

Fetching the angles in file efg.{}.out
skipped:696
skipped:812
skipped:813
skipped:814
skipped:815
skipped:816
skipped:817
skipped:818
skipped:819
skipped:820
skipped:821
skipped:822
skipped:823
skipped:824
skipped:825
skipped:826
skipped:827
skipped:828
skipped:829
skipped:830
skipped:831
skipped:832
skipped:833
skipped:834
skipped:835
skipped:836
skipped:837
skipped:838
skipped:839
skipped:840
skipped:841
skipped:842
skipped:843
skipped:844
skipped:845
skipped:846
skipped:847
skipped:848
skipped:849
skipped:850
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipped:857
skipped:858
skipped:859
skipped:860
skipped:861
skipped:862
skipped:863
skipped:864
skipped:865
skipped:866
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
s

In [3]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

target_atom_labels=	[1, 3, 4, 6, 23, 25, 26, 28, 37, 39, 40, 42, 51, 53, 54, 56]

av_theta_xs=	[-0.010059767492857519, -0.0070465791091563514, 0.014828528493156327, -0.0084360592626390677, -0.042197051610639809, -0.0036087691109035311, 0.018511297096804782, 0.013141041856401794, 0.0037297525863728576, 0.020492343868304872, -0.0047569741590413874, -0.015737032618317253, -0.037154932853015563, -0.0029627886951387795, 0.008981521106539889, 0.016325457257319629]
    
av_theta_ys=	[0.0025846118516739966, 0.0019425657325231633, -0.017467421310842023, -0.0035491240348430111, 0.00050427144772768374, -0.0021469756402791003, -0.00074162155249078232, -0.0015888925538200599, -0.0074147688492273139, -0.014410877061182943, -0.0055152094930808505, 0.0044503200942530114, 0.0033561169985523334, 0.0015488125737915146, -0.0014167953510286406, 0.0043818925535916839]
  
av_theta_y2s=	[0.0019158634319037101, 0.0068689427078310241, 0.001753753606017442, 0.0052303328282336456, 0.0048781294995730801, 0.0013134334795225563, 0.0044230442189515792, 0.0021516988605096194, 0.008371227273873556, 0.0013998405956914398, 0.0081027360461240101, 0.0019973662184875015, 0.003315306664087895, 0.0020399243097013846, 0.0053499875353373028, 0.0021041281417571809]

av_theta_x2s=	[0.0016680539190516918, 0.015741324244749121, 0.0016673194467981092, 0.015505265267014911, 0.016679368524056184, 0.0025731140619503563, 0.012111219771367992, 0.0016741820856230893, 0.017702057633629257, 0.0016841550996280167, 0.014164213812446515, 0.0027800787506590476, 0.017158199352996076, 0.0014265020749073846, 0.010647877609003459, 0.0019798594085857265]
 
cq_coef0s=	[0.99462412397356692, 0.9660845995711298, 0.99486839042077668, 0.9688966028571272, 0.96766375296455609, 0.99417017868779067, 0.97519860401452063, 0.99426117858080099, 0.96089007263874582, 0.99537400645702079, 0.96659957521214424, 0.99283383254628021, 0.96928974097437404, 0.99480036042308684, 0.9760032022834888, 0.99387401867448566]

cq_coef1s=	[0.99452914726062369, 0.97012401743725785, 0.99483769548998391, 0.97315712928019893, 0.97255743175703369, 0.99461752275462534, 0.97869893424134147, 0.99407816311407404, 0.96475912477449444, 0.9954749736452313, 0.96935924480959557, 0.99313381886492291, 0.97503004250014702, 0.99458251885195559, 0.97841526541040313, 0.99382639081846669]

eta_coefs=	[]

cq0s=	[-6.2181, -0.7353, -6.0769, -0.8046, -0.8046, -6.077, -0.7353, -6.2181, -0.8046, -6.0769, -0.7353, -6.2181, -0.8046, -6.077, -0.7353, -6.2181]

cqprime0s=	[-6.1846722652800361, -0.71036200606465172, -6.0457157217480182, -0.77957420665884447, -0.77858225563528183, -6.0415721758857037, -0.71706353353187702, -6.1824154345332785, -0.77313215244513489, -6.0487882998386695, -0.71074066765348964, -6.1735400541560246, -0.77989052558798133, -6.0454017902910984, -0.71765515463904928, -6.180008035519819]

cqprime1s=	[-6.1840816905812837, -0.71333219002161563, -6.0455291917230838, -0.78300222621884807, -0.78251970959170924, -6.044290685779858, -0.71963732634765831, -6.1812774260596237, -0.77624519179355822, -6.0494018673447059, -0.71276985270849558, -6.175405399083977, -0.78450917219561822, -6.044077967063334, -0.71942874465626938, -6.1797118807483074]
 
eta0s=	[0.51102, 0.60704, 0.4735, 0.55287, 0.5529, 0.4735, 0.60705, 0.51102, 0.55287, 0.4735, 0.60704, 0.51102, 0.5529, 0.4735, 0.60705, 0.51102]

etaprimes=	[]

fq0s=  [np.array([-1.58024561, -3.84838139, -5.428627  ]),np.array([-0.21560908, -0.42496697, -0.64057604]),np.array([-1.4313232 , -3.81862519, -5.24994839]),np.array([-0.2155016 , -0.47692986, -0.69243145]),np.array([-0.21523906, -0.47631716, -0.69155622]),np.array([-1.43034221, -3.81600803, -5.24635024]),np.array([-0.21764671, -0.4289743 , -0.646621  ]),np.array([-1.57966897, -3.84697709, -5.42664606]),np.array([-0.21372079, -0.47298872, -0.68670951]),np.array([-1.43205063, -3.82056591, -5.25261654]),np.array([-0.21572401, -0.4251935 , -0.6409175 ]),np.array([-1.57740122, -3.84145443, -5.41885565]),np.array([-0.21560074, -0.47711753, -0.69271826]),np.array([-1.43124887, -3.81842691, -5.24967578]),np.array([-0.21782628, -0.42932823, -0.64715451]),np.array([-1.57905385, -3.8454791 , -5.42453295])]

fqprimes=	[]
